In [1]:
import os
import sys
import numpy as np
import pandas as pd

model01 = pd.read_csv("../model02/oof/oof.csv").rename(columns={"sii": "sii01"})
model02 = pd.read_csv("../model02/oof/oof.csv").rename(columns={"sii": "sii02"})
model03 = pd.read_csv("../model03/oof/oof.csv").rename(columns={"pred_sii": "sii03"})
model04 = pd.read_csv("../model04/oof/oof.csv").rename(columns={"pred_sii": "sii04"})
model05 = pd.read_csv("../model05/oof/oof.csv").rename(columns={"sii": "sii05"})
# model06 = pd.read_csv("../model06/oof/oof.csv").rename(columns={"sii": "sii06"})
model06 = pd.read_csv("../model03/oof/oof.csv").rename(columns={"pred_sii": "sii06"})

gt = pd.read_csv("../../../inputs/train.csv")
gt = gt[["id", "sii"]].dropna()

gt = (
    gt.merge(model01, on="id")
    .merge(model02, on="id")
    .merge(model03, on="id")
    .merge(model04, on="id")
    .merge(model05, on="id")
    .merge(model06, on="id")
)
gt

,id,sii,sii01,sii02,sii03,sii04,sii05,sii06
0,00008ff9,2.0,0.414452,0.414452,0.325819,0.548154,0.258869,0.325819
1,000fd460,0.0,0.215259,0.215259,0.275750,0.337649,0.170646,0.275750
2,00105258,0.0,0.622507,0.622507,1.044493,0.955384,0.544412,1.044493
3,00115b9f,1.0,0.258004,0.258004,0.413800,0.387743,0.179771,0.413800
4,001f3379,1.0,0.434354,0.434354,0.357467,0.222558,0.355132,0.357467
...,...,...,...,...,...,...,...,...
2731,ff6c2bb8,0.0,0.444530,0.444530,0.547489,0.297347,0.446794,0.547489
2732,ff759544,1.0,0.181993,0.181993,0.234311,0.251396,0.111413,0.234311
2733,ff8a2de4,1.0,0.780605,0.780605,0.969899,0.664553,0.827977,0.969899
2734,ffcd4dbd,1.0,0.772518,0.772518,0.943198,0.867243,0.707650,0.943198


***平均値アンサンブル***

In [2]:
# 平均値
from sklearn.metrics import *
from scipy.optimize import minimize


def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights="quadratic")


def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(
        oof_non_rounded < thresholds[0],
        0,
        np.where(
            oof_non_rounded < thresholds[1],
            1,
            np.where(oof_non_rounded < thresholds[2], 2, 3),
        ),
    )


def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    mean_weight = thresholds[:6]
    thresholds = thresholds[6:]

    model01_pred = oof_non_rounded[:, 0]
    model02_pred = oof_non_rounded[:, 1]
    model03_pred = oof_non_rounded[:, 2]
    model04_pred = oof_non_rounded[:, 3]
    model05_pred = oof_non_rounded[:, 4]
    model06_pred = oof_non_rounded[:, 5]
    oof_non_rounded = (
        mean_weight[0] * model01_pred
        + mean_weight[1] * model02_pred
        + mean_weight[2] * model03_pred
        + mean_weight[3] * model04_pred
        + mean_weight[4] * model05_pred
        + mean_weight[5] * model06_pred
    )
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)


pred = gt[["sii01", "sii02", "sii03", "sii04", "sii05", "sii06"]].values
# pred = pred[:, 0] * (1 / 3) + pred[:, 1] * (1 / 3) + pred[:, 2] * (1 / 3)
pred = (
    pred[:, 0] * (1 / 6)
    + pred[:, 1] * (1 / 6)
    + pred[:, 2] * (1 / 6)
    + pred[:, 3] * (1 / 6)
    + pred[:, 4] * (1 / 6)
    + pred[:, 5] * (1 / 6)
)

oof_tuned = threshold_Rounder(pred, [0.5, 1.5, 2.5])
tKappa = quadratic_weighted_kappa(gt["sii"], oof_tuned)

print(f"tuned Kappa: {tKappa:.3f}")

tuned Kappa: 0.399


***重みもNelder-Mead***

In [3]:
gt

,id,sii,sii01,sii02,sii03,sii04,sii05,sii06
0,00008ff9,2.0,0.414452,0.414452,0.325819,0.548154,0.258869,0.325819
1,000fd460,0.0,0.215259,0.215259,0.275750,0.337649,0.170646,0.275750
2,00105258,0.0,0.622507,0.622507,1.044493,0.955384,0.544412,1.044493
3,00115b9f,1.0,0.258004,0.258004,0.413800,0.387743,0.179771,0.413800
4,001f3379,1.0,0.434354,0.434354,0.357467,0.222558,0.355132,0.357467
...,...,...,...,...,...,...,...,...
2731,ff6c2bb8,0.0,0.444530,0.444530,0.547489,0.297347,0.446794,0.547489
2732,ff759544,1.0,0.181993,0.181993,0.234311,0.251396,0.111413,0.234311
2733,ff8a2de4,1.0,0.780605,0.780605,0.969899,0.664553,0.827977,0.969899
2734,ffcd4dbd,1.0,0.772518,0.772518,0.943198,0.867243,0.707650,0.943198


In [4]:
init_thresholds = gt.copy()
init_thresholds["mean"] = gt[
    ["sii01", "sii02", "sii03", "sii04", "sii05", "sii06"]
].mean(axis=1)
init_thresholds = (
    init_thresholds.groupby(["sii"])["mean"]
    .mean()
    .reset_index()
    .iloc[1:]["mean"]
    .values.tolist()
)

KappaOPtimizer = minimize(
    evaluate_predictions,
    x0=[1 / 6, 1 / 6, 1 / 6, 1 / 6, 1 / 6, 1 / 6] + init_thresholds,
    args=(gt["sii"], gt[["sii01", "sii02", "sii03", "sii04", "sii05", "sii06"]].values),
    method="Nelder-Mead",
    options={"maxiter": 10000},
)

pred = gt[["sii01", "sii02", "sii03", "sii04", "sii05", "sii06"]].values
pred = (
    pred[:, 0] * KappaOPtimizer.x[0]
    + pred[:, 1] * KappaOPtimizer.x[1]
    + pred[:, 2] * KappaOPtimizer.x[2]
    + pred[:, 3] * KappaOPtimizer.x[3]
    + pred[:, 4] * KappaOPtimizer.x[4]
    + pred[:, 5] * KappaOPtimizer.x[5]
)

oof_tuned = threshold_Rounder(pred, KappaOPtimizer.x[6:])
tKappa = quadratic_weighted_kappa(gt["sii"], oof_tuned)

print(f"tuned Kappa: {tKappa:.3f}")  # tuned Kappa: 0.454

tuned Kappa: 0.468


In [5]:
KappaOPtimizer.x

array([0.16195663, 0.15049577, 0.17038008, 0.17680231, 0.15660695,
       0.16357144, 0.53886885, 1.02527822, 1.54556097])

***LGBMでStacking***

In [6]:
# 回帰モデルの作成
# 教師:"sii", 説明変数: "sii01", "sii02", "sii03", "sii04", "sii05"

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# 決定木モデルの作成
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor

from sklearn.model_selection import StratifiedKFold

# rasso回帰
from sklearn.linear_model import Lasso

# model = LinearRegression()
# model.fit(gt[["sii01", "sii02", "sii03", "sii04", "sii05"]], gt["sii"])
# # 予測
# gt["pred_sii"] = model.predict(gt[["sii01", "sii02", "sii03", "sii04", "sii05"]])

# 重みの確認
# list(model.coef_)

# 決定木モデル
# model = DecisionTreeClassifier(max_depth=6)
# model.fit(gt[["sii01", "sii02", "sii03", "sii04", "sii05"]], gt["sii"])
# # 予測
# gt["pred_sii"] = model.predict(gt[["sii01", "sii02", "sii03", "sii04", "sii05"]])

# LGBM

skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=2025)
for t_index, v_index in skf.split(
    gt[["sii01", "sii02", "sii03", "sii04", "sii05"]], gt["sii"]
):
    pass

train_lgb = gt.iloc[t_index].reset_index(drop=True)
valid_lgb = gt.iloc[v_index].reset_index(drop=True)

# model = LGBMRegressor()
# model = DecisionTreeRegressor(max_depth=6)
# model = LinearRegression()
model = Lasso(alpha=0.1)
model.fit(train_lgb[["sii01", "sii02", "sii03", "sii04", "sii05"]], train_lgb["sii"])
# 予測
valid_lgb["pred_sii"] = model.predict(
    valid_lgb[["sii01", "sii02", "sii03", "sii04", "sii05"]]
)
gt = pd.concat([valid_lgb], axis=0).reset_index(drop=True)
gt.head()

,id,sii,sii01,sii02,sii03,sii04,sii05,sii06,pred_sii
0,000fd460,0.0,0.215259,0.215259,0.275750,0.337649,0.170646,0.275750,0.490356
1,00105258,0.0,0.622507,0.622507,1.044493,0.955384,0.544412,1.044493,0.711742
2,001f3379,1.0,0.434354,0.434354,0.357467,0.222558,0.355132,0.357467,0.511805
3,00ae59c9,1.0,0.814845,0.814845,0.650288,0.777258,0.720626,0.650288,0.599816
4,00c0cd71,2.0,0.449843,0.449843,0.680836,0.553055,0.514491,0.680836,0.605745


In [7]:
from sklearn.metrics import *
from scipy.optimize import minimize


def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights="quadratic")


def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(
        oof_non_rounded < thresholds[0],
        0,
        np.where(
            oof_non_rounded < thresholds[1],
            1,
            np.where(oof_non_rounded < thresholds[2], 2, 3),
        ),
    )


def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    thresholds = thresholds

    oof_non_rounded = oof_non_rounded
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)


KappaOPtimizer = minimize(
    evaluate_predictions,
    x0=[0.5, 1.5, 2.5],
    args=(gt["sii"], gt[["pred_sii"]].values),
    method="Nelder-Mead",
    options={"maxiter": 10000},
)

pred = gt[["pred_sii"]].values

oof_tuned = threshold_Rounder(pred, KappaOPtimizer.x)
tKappa = quadratic_weighted_kappa(gt["sii"], oof_tuned)

print(f"tuned Kappa: {tKappa:.3f}")  # tuned Kappa: 0.454

tuned Kappa: 0.387


In [8]:
KappaOPtimizer.x

array([0.56135795, 1.42017211, 2.37931349])

In [9]:
KappaOPtimizer.x[3:]

array([], dtype=float64)